In [1]:
import pandas as pd

df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/26/problem7.csv')
df.head()

,height,weight,waistline
0,174.396,72.102,79.3787
1,179.656,81.255,80.6649
2,175.079,76.207,80.3166
3,180.804,81.354,80.8794
4,177.448,78.768,80.3499


In [19]:
import jax.numpy as jnp
from jax import random

import numpy as np

from numpyro import sample, plate
from numpyro.infer import MCMC, NUTS
import numpyro.distributions as dist

rng_key = random.PRNGKey(1234) # 시드고정

def model(X, y): # y_est = alpha * x + beta # 모델 구축
    # 사전 분포
    beta = sample('beta', dist.ImproperUniform(support=dist.constraints.real,
                                           batch_shape=jnp.shape(X)[1:], event_shape=()))
    sigma = sample('sigma', dist.InverseGamma(concentration=0.005, rate=0.005)) # 역감마 분포 - 오차항의 분산의 사전 분포
    alpha = sample('alpha', dist.ImproperUniform(support=dist.constraints.real, batch_shape=(1,), event_shape=()))
    
    # 표본 분포 관측치의 가능도(Likelihood)
    mu = jnp.dot(X, beta) + alpha
    with plate('data', len(y)):
        sample('obs', dist.Normal(mu, sigma), obs=y)

In [20]:
X = df.iloc[:, [0, 2]].values
y = df.iloc[:, 1].values

In [21]:
# MCMC
kernel = NUTS(model)
mcmc = MCMC(kernel, num_warmup=1000, num_samples=10000) # num_warmup : burn-in, num_samples : 샘플 개수
mcmc.run(rng_key, X, y)
mcmc.print_summary()

sample: 100%|██████████████████████| 11000/11000 [00:24<00:00, 450.38it/s, 1023 steps of size 1.64e-03. acc. prob=0.94]



                mean       std    median      5.0%     95.0%     n_eff     r_hat
  alpha[0]   -124.58     30.44   -124.33   -175.86    -76.39   1008.28      1.00
   beta[0]      1.04      0.06      1.04      0.94      1.13   1450.26      1.00
   beta[1]      0.23      0.46      0.22     -0.53      0.96    960.88      1.00
     sigma      1.69      0.06      1.69      1.59      1.78   2531.83      1.00

Number of divergences: 0


In [28]:
posterior_samples = mcmc.get_samples()
beta_samples = posterior_samples['beta']
alpha_samples = posterior_samples['alpha']
sigma_samples = posterior_samples['sigma']

print(f'beta {np.mean(beta_samples[:, 0]):.2f} (height)\nbeta {np.mean(beta_samples[:, 1]):.2f} (waistline)')
print(f'alpha {np.mean(alpha_samples):.2f}')

beta 1.04 (height)
beta 0.23 (waistline)
alpha -124.58


In [29]:
input_data = np.array([[180.0, 85.0]], dtype='float32')

predicted_weights = np.dot(input_data, beta_samples.T) + alpha_samples
predicted_weights = np.mean(predicted_weights, axis=0)

print(f'Predicted weight : {np.mean(predicted_weights):.3f}')

Predicted weight : 81.088
